## [K-TACC 프로젝트](https://github.com/kyle-bong/K-TACC) 소스코드를 활용한 데이터 증강

- 문맥을 고려한 한국어 텍스트 데이터 증강 (Korean Text Augmentation Considering Context, K-TACC)

| Model | Pearson's correlation  |
|--------|--------|
| BertAugmentation (Random Masking Insertion)    | 0.9300    |
| BertAugmentation (Random Masking Replacement)    | 0.9023    |

In [1]:
import pandas as pd
from utils.BertAugmentation import BertAugmentation
from utils.adverb_augmentation import AdverbAugmentation
import transformers

/home/cho/anaconda3/envs/augmen/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
BERT_aug = BertAugmentation()
random_masking_replacement = BERT_aug.random_masking_replacement
random_masking_insertion = BERT_aug.random_masking_insertion

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


## BERT augmentation

In [3]:
sentence = "아버지가 오늘 아침에 부엌에 있었다가 갑자기 일어나시더니 슬그머니 안방으로 들어가셨다."
ratio = 0.15

In [4]:
print('random_masking_replacement:', random_masking_replacement(sentence, ratio))
print('random_insertion_replacement:', random_masking_insertion(sentence, ratio))

random_masking_replacement: 아버지가 잠시 부엌에 있었다가 갑자기 일어나시더니 슬그머니 안방으로 들어가셨다.
random_insertion_replacement: 아버지가 오늘 아침에는 부엌에 와 있었다가 갑자기 일어나시더니 슬그머니 안방으로 들어가셨다.


In [5]:
train_df = pd.read_csv('/home/cho/test/aiffel-dl-thon-dktc-online-12/train.csv')
train_df.head()

,idx,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [6]:
normal_df = pd.read_csv('/home/cho/test/aiffel-dl-thon-dktc-online-12/일반대화 합성데이터(GPT-4o, AIhub 참고).csv')
normal_df.head()

,idx,class,conversation
0,0,일반 대화,이번에 캘리포니아에 산불 난 거 보셨어요?\n네 봤어요 ㅜㅜ 일주일 넘게 진압하고 ...
1,1,일반 대화,얼마 전에 동네 피부 미용샵에 다녀왔어요! 여기 좋더라고요!\n동네에 그런 곳이 있...
2,2,일반 대화,코로나 터지고 건강에 더 주의를 기울이게 된 것 같아요\n인정이요. 위생에도 더 철...
3,3,일반 대화,"요즘 택배가 참 편하긴 한 것 같아요\n맞아요, 옛날엔 몰랐는데 새벽 배송에 로켓 ..."
4,4,일반 대화,다들 TV프로그램 뭐 보세요?\n요즘 볼게 없어요\n전 슬기로운 의사생활 재밌게 봤...


In [7]:
# train data 와 normal 데이터 합치기
def concat_train_normal(train_df, normal_df):
    train_df = pd.concat([train_df, normal_df], ignore_index=True)
    train_df.drop(['idx'], axis=1, inplace=True)
    train_df.reset_index(inplace=True)

    return train_df

train_df = concat_train_normal(train_df, normal_df)
train_df.head()

,index,class,conversation
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [8]:
# 클래스 숫자로 변환
def class_encoding(train_df, class_mapping):
    train_df['class_encoded'] = train_df['class'].replace(class_mapping)
    
    return train_df

# 클래스 매핑 딕셔너리 정의
class_mapping = {
    "협박 대화": 0,
    "갈취 대화": 1,
    "직장 내 괴롭힘 대화": 2,
    "기타 괴롭힘 대화": 3,
    "일반 대화": 4
}

train_df = class_encoding(train_df, class_mapping)

# 변환된 데이터 확인
train_df.head()


,index,class,conversation,class_encoded
0,0,협박 대화,지금 너 스스로를 죽여달라고 애원하는 것인가?\n 아닙니다. 죄송합니다.\n 죽을 ...,0
1,1,협박 대화,길동경찰서입니다.\n9시 40분 마트에 폭발물을 설치할거다.\n네?\n똑바로 들어 ...,0
2,2,기타 괴롭힘 대화,너 되게 귀여운거 알지? 나보다 작은 남자는 첨봤어.\n그만해. 니들 놀리는거 재미...,3
3,3,갈취 대화,어이 거기\n예??\n너 말이야 너. 이리 오라고\n무슨 일.\n너 옷 좋아보인다?...,1
4,4,갈취 대화,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...,1


In [9]:
# 중복된 'conversation' 제거
train_df = train_df.drop_duplicates(subset=['conversation'])

# 중복 제거 후 데이터 확인
print(f"중복 제거된 후 데이터 수: {train_df.shape[0]}")

중복 제거된 후 데이터 수: 4846


In [10]:
train_df = train_df.reset_index(drop=True)

In [11]:
train_df.to_csv('raw_data.csv',index=False)

In [12]:
CLASS = train_df.loc[392,'class']
class_encoded = train_df.loc[392,'class_encoded']
conversation = train_df.loc[392,'conversation']

replaced_text = random_masking_replacement(conversation, ratio)
inserted_text = random_masking_insertion(conversation, ratio)

new_data_replaced = {'class' : CLASS, 
            'class_encoded' : class_encoded, 
            'conversation' : replaced_text
           }

new_data_inserted = {'class' : CLASS, 
            'class_encoded' : class_encoded, 
            'conversation' : inserted_text
           }

print(new_data_replaced)

{'class': '직장 내 괴롭힘 대화', 'class_encoded': 2, 'conversation': '회식 갈거지? 오늘 몸이 좀 안 아파 오늘 왜 잘 못해. 한다고 하는데 뜻대로 안되네요. 오늘 회식은 과장님 축하 위한거니 되도록이면 모두 참석 했으면 하는데. 죄송합니다. 그 정도로 될 것 같지 않은데 잠깐이라도 참석해서 더 이상 안나오게 해. 잘 알겠습니다. 사람이 참 융통성이 없어. 죄송합니다.'}


In [13]:
import pandas as pd

# 결과 저장을 위한 리스트
replaced_text_data = []
inserted_text_data = []

# 비율 설정
ratio = 0.15

# 모든 행에 대해 데이터 변환 수행
for i in range(len(train_df)):
    try:
        cls = train_df.loc[i, 'class']
        class_encoded = train_df.loc[i, 'class_encoded']
        conversation = train_df.loc[i, 'conversation']
        
        replaced_text = random_masking_replacement(conversation, ratio)
        inserted_text = random_masking_insertion(conversation, ratio)
        
        # 변형된 데이터 추가
        replaced_text_data.append({'class': cls, 'class_encoded': class_encoded, 'conversation': replaced_text})
        inserted_text_data.append({'class': cls, 'class_encoded': class_encoded, 'conversation': inserted_text})
    
    except Exception as e:
        print(f"Error at index {i}: {e}")
        continue  # 오류 발생 시 해당 인덱스를 건너뛰고 다음으로 진행

# 변형된 데이터를 DataFrame으로 변환
augmented_replaced_text_data = pd.DataFrame(replaced_text_data)
augmented_inserted_text_data = pd.DataFrame(inserted_text_data)

augmented_replaced_text_data.head()

Token indices sequence length is longer than the specified maximum sequence length for this model (513 > 512). Running this sequence through the model will result in indexing errors


Error at index 3925: The size of tensor a (513) must match the size of tensor b (512) at non-singleton dimension 1
Error at index 3957: The size of tensor a (545) must match the size of tensor b (512) at non-singleton dimension 1
Error at index 3960: The size of tensor a (524) must match the size of tensor b (512) at non-singleton dimension 1
Error at index 4076: The size of tensor a (513) must match the size of tensor b (512) at non-singleton dimension 1
Error at index 4078: The size of tensor a (609) must match the size of tensor b (512) at non-singleton dimension 1
Error at index 4305: The size of tensor a (540) must match the size of tensor b (512) at non-singleton dimension 1
Error at index 4310: The size of tensor a (513) must match the size of tensor b (512) at non-singleton dimension 1
Error at index 4378: The size of tensor a (513) must match the size of tensor b (512) at non-singleton dimension 1
Error at index 4380: The size of tensor a (513) must match the size of tensor b 

,class,class_encoded,conversation
0,협박 대화,0,지금 너를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면 혼자 죽지...
1,협박 대화,0,길동경찰서입니다. 9시 40분 마트에 폭발물을 설치할거다. 네? 똑바로 들어 한번만...
2,기타 괴롭힘 대화,3,너 되게 귀여운거? 나보다 작은 남자는 첨봤어. 그만해. 너 놀리는거 재미없어. 지...
3,갈취 대화,1,어이 거기 예?? 그 말이야. 이리 와서 무슨 일. 너 옷 좋아보인다? 얘 돈 좀....
4,갈취 대화,1,저기요 혹시 날이 너무 뜨겁잖아요? 저희 회사에서 이 선크림 파는데 한 번 손등에 ...


In [14]:
augmented_inserted_text_data.head()

,class,class_encoded,conversation
0,협박 대화,0,왜 지금 너 스스로를 죽여달라고 애원하는 것인가? 아닙니다. 죄송합니다. 죽을 거면...
1,협박 대화,0,길동경찰서입니다. 9시 40분 마트에 폭발물을 하나 더 설치할거다.? 네? 똑바로 ...
2,기타 괴롭힘 대화,3,너 되게 귀여운거 알지? 나보다 키 작은 남자는 첨봤어. 그만해. 니들 놀리는거 진...
3,갈취 대화,1,어이 거기 예?? 너 말이야 너. 너 이리 오라고 무슨 일. 너 옷 좋아보인다? 얘...
4,갈취 대화,1,저기요 혹시 날이 너무 많이 뜨겁잖아요? 저희 회사에서 지금 이 선크림 파는데 한 ...


In [ ]:
augmented_replaced_text_data.to_csv('augmented_replaced_text_data.csv', index=False)
augmented_inserted_text_data.to_csv('augmented_inserted_text_data.csv', index=False)